In [ ]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler, StandardScaler


In [ ]:
raw_data = pd.read_csv(r"../data/df_train.csv", index_col=0)
raw_data.dropna(axis=1, inplace=True)


In [ ]:
us_data = pd.DataFrame()

defualt = raw_data[raw_data["TARGET"] == 1]

paid = raw_data[raw_data["TARGET"] == 0]
paid = paid.sample(len(defualt), random_state=42)

us_data = pd.concat([defualt, paid], axis=0)
us_data = us_data.sample(frac=1)

us_data.shape

In [ ]:
x = us_data.drop(["TARGET"], axis=1)
y = us_data.filter(["TARGET"], axis=1)

scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)
y_scaled = y.values

x_scaled.shape, y_scaled.shape

In [ ]:
y_scaled.dtype

In [ ]:
import tensorflow as tf
import keras
from keras import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.initializers import GlorotUniform
import keras_tuner as kt
from kerastuner import HyperParameters as hp


In [ ]:
def model_builder(hp):
    model = Sequential()

    # input layer
    model.add(Dense(units=x_scaled.shape[1], input_shape=(x_scaled.shape[1],)))

    # let the model decide how many layers it wants to have
    for i in range(hp.Int("num_layers", min_value=2, max_value=5, step=1)):
        model.add(
            Dropout(hp.Float("dropout_" + str(i), min_value=0, max_value=0.5, step=0.1))
        )
        model.add(
            Dense(
                units=hp.Int("layer_" + str(i), min_value=32, max_value=1024, step=64),
                activation=hp.Choice("act_" + str(i), ["relu", "sigmoid"]),
            )
        )

    # output shape of the model the same as the number of features
    model.add(
        Dense(
            (1),
            activation="sigmoid"),
        )

    hp_learning_rate = hp.Choice("learning_rate", values=[1e-1, 1e-2, 1e-3, 1e-4])
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
        loss=keras.losses.BinaryCrossentropy(),
        metrics=["accuracy"],
    )

    return model

In [ ]:
earlyStop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=4)

In [ ]:
tuner = kt.BayesianOptimization(
    model_builder,
    objective=kt.Objective("accuracy", direction="max"),
    directory=r"../logs",
    project_name="BayOpt_v1.01",
)

In [ ]:
tuner.search(
    x_scaled,
    y_scaled,
    epochs=20,
    batch_size=32,
    validation_split=0.2,
    callbacks=[earlyStop],
)

In [ ]:
hpModel = tuner.get_best_models()[0]
hpModel.summary()


In [ ]:
history = hpModel.fit(x_scaled, y_scaled, epochs=30, batch_size=32, callbacks=earlyStop)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameter

# Load and preprocess the data
(X_train, y_train), (X_test, y_test) = keras.datasets.creditcard_default.load_data()
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
y_train = y_train.reshape(y_train.shape[0], 1)
y_test = y_test.reshape(y_test.shape[0], 1)
input_shape = (X_train.shape[1], 1)

# Define the model-building function
def build_model(hp):
    model = keras.Sequential()
    # Add convolutional layers
    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(layers.Conv1D(filters=hp.Int('filters_' + str(i), 16, 256, 16),
                                kernel_size=hp.Choice('kernel_size_' + str(i), [3, 5, 7]),
                                activation='relu',
                                input_shape=input_shape))
        model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Flatten())
    # Add dense layers
    for i in range(hp.Int('num_dense_layers', 1, 3)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i), 32, 512, 32), activation='relu'))
        model.add(layers.Dropout(rate=hp.Float('dropout_rate_' + str(i), 0, 0.5, 0.1)))
    model.add(layers.Dense(units=1, activation='sigmoid'))
    # Compile the model
    model.compile(optimizer=keras.optimizers.Adam(hp.Float('learning_rate', 0.0001, 0.01, sampling='log')),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

# Define the hyperparameters to search over
tuner = RandomSearch(build_model,
                     objective='val_accuracy',
                     max_trials=10,
                     directory='my_dir',
                     project_name='credit_default_classification')

# Search for the best hyperparameters
tuner.search(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Print the best hyperparameters
best_hyperparameters = tuner.get_best_hyperparameters(1)[0]
print('Best hyperparameters:', best_hyperparameters)

# Build and train the model with the best hyperparameters
model = tuner.hypermodel.build(best_hyperparameters)
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print('Test accuracy:', accuracy)
